<a href="https://colab.research.google.com/github/takzen/ai-engineering-handbook/blob/main/60_Bayesian_Optimization_Optuna.ipynb" target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# 🎯 Bayesian Optimization: Inteligentne strojenie (Optuna)

Grid Search jest głupi – sprawdza kombinacje, które nie mają sensu.
Random Search jest okej, ale to hazard.

**Optymalizacja Bayesowska** buduje probabilistyczny model (tzw. Surrogate Model) Twojej funkcji celu.
Uczy się na bieżąco: *"Jeśli `learning_rate=0.1` dał słaby wynik, to nie sprawdzaj `0.11`, sprawdź `0.001`"*.

Użyjemy biblioteki **Optuna**, która implementuje algorytm **TPE (Tree-structured Parzen Estimator)**. Jest szybka, prosta i sama rysuje wykresy.

**Zadanie:** Znaleźć idealne parametry dla Lasu Losowego na trudnym zbiorze danych.

In [1]:
# Instalacja (jeśli nie masz w środowisku)
# !uv pip install optuna

import optuna
import sklearn
import sklearn.datasets
import sklearn.ensemble
import sklearn.model_selection
import sklearn.svm

# 1. DANE (Rak piersi - klasyfikacja)
data = sklearn.datasets.load_breast_cancer()
X, y = data.data, data.target

print(f"Dane wczytane. Kształt: {X.shape}")

Dane wczytane. Kształt: (569, 30)


## Funkcja Celu (Objective Function)

To jest serce Optuny.
Definiujemy funkcję, która:
1.  Przyjmuje obiekt `trial` (próbę).
2.  Prosi `trial` o sugestię parametrów (np. *"Wylosuj mi liczbę całkowitą z zakresu 2-32"*).
3.  Trenuje model z tymi parametrami.
4.  Zwraca wynik (np. Accuracy).

Optuna będzie wywoływać tę funkcję wielokrotnie, za każdym razem podsuwając "mądrzejsze" parametry.

In [2]:
def objective(trial):
    # 1. Sugerowanie parametrów (Search Space)
    
    # Wybierz model (Ciekawostka: Optuna może dobierać nawet typ modelu!)
    classifier_name = trial.suggest_categorical("classifier", ["SVC", "RandomForest"])
    
    if classifier_name == "SVC":
        # Parametry dla SVM
        svc_c = trial.suggest_float("svc_c", 1e-10, 1e10, log=True) # Skala logarytmiczna!
        model = sklearn.svm.SVC(C=svc_c, gamma="auto")
        
    else:
        # Parametry dla Lasu
        rf_n_estimators = trial.suggest_int("rf_n_estimators", 10, 1000)
        rf_max_depth = trial.suggest_int("rf_max_depth", 2, 32, log=True)
        model = sklearn.ensemble.RandomForestClassifier(
            n_estimators=rf_n_estimators, 
            max_depth=rf_max_depth
        )

    # 2. Walidacja Krzyżowa (3-Fold)
    # Żeby wynik był wiarygodny, robimy szybkie CV
    score = sklearn.model_selection.cross_val_score(model, X, y, n_jobs=-1, cv=3)
    accuracy = score.mean()
    
    return accuracy

print("Funkcja celu zdefiniowana.")

Funkcja celu zdefiniowana.


## Uruchomienie Badania (Study)

Tworzymy obiekt `study` i mówimy mu: *"Maksymalizuj wynik funkcji objective"*.
Damy mu 50 prób (`n_trials=50`).

Obserwuj logi. Zobaczysz, jak Optuna skacze po parametrach.

In [3]:
# Wyłączamy nadmiar logów (żeby było czytelniej)
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Tworzymy badanie
study = optuna.create_study(direction="maximize")

print("🚀 Start optymalizacji...")
# Uruchamiamy (50 prób)
study.optimize(objective, n_trials=50, show_progress_bar=True)

print("✅ Koniec.")

🚀 Start optymalizacji...


  0%|          | 0/50 [00:00<?, ?it/s]

✅ Koniec.


## Analiza Wyników

Co wygrało? Czy lepszy był SVM czy Las? Jakie parametry zadziałały?

In [4]:
print("🏆 NAJLEPSZY WYNIK:")
print(f"Accuracy: {study.best_value:.4f}")
print("Parametry:")
for key, value in study.best_params.items():
    print(f"  {key}: {value}")

# Sprawdźmy historię
trials_df = study.trials_dataframe()
# Sortujemy po wyniku
display(trials_df.sort_values('value', ascending=False).head(5)[['params_classifier', 'value']])

🏆 NAJLEPSZY WYNIK:
Accuracy: 0.9631
Parametry:
  classifier: RandomForest
  rf_n_estimators: 585
  rf_max_depth: 17


,params_classifier,value
11,RandomForest,0.963102
22,RandomForest,0.963102
24,RandomForest,0.963093
23,RandomForest,0.961348
21,RandomForest,0.961339


In [5]:
# WIZUALIZACJA 1: Historia Optymalizacji
# Zobaczysz, czy algorytm się "uczył" (czy punkty pną się w górę)
optuna.visualization.plot_optimization_history(study).show()

In [6]:
# WIZUALIZACJA 2: Ważność Hiperparametrów
# Co miało największy wpływ na wynik? Typ modelu? Głębokość drzewa?
try:
    fig = optuna.visualization.plot_param_importances(study)
    fig.show()
except:
    print("Potrzeba więcej prób, aby ocenić ważność parametrów dla różnych modeli.")

## 🧠 Podsumowanie: Dlaczego Optuna?

1.  **Warunkowość:** Zauważ, że w kodzie użyliśmy `if classifier == "SVC"`.
    *   GridSearch by zgłupiał (próbowałby ustawić `rf_n_estimators` dla SVM-a).
    *   Optuna rozumie, że parametr `rf_n_estimators` istnieje TYLKO wtedy, gdy wybrano Las. To pozwala testować **różne architektury** w jednym przebiegu.
2.  **Pruning (Przycinanie):** Optuna potrafi przerwać trening w połowie (np. po 10 epokach sieci neuronowej), jeśli widzi, że "nic z tego nie będzie". (Tu tego nie użyliśmy, ale przy Deep Learningu to oszczędza 50% czasu).

**Wniosek:**
Przestań używać `GridSearchCV`. Zacznij używać Optuny. Jest szybsza, mądrzejsza i obsługuje PyTorch/XGBoost/Sklearn w jednym standardzie.